In [ ]:
import rioxarray
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
import glob
from osgeo import gdal

%matplotlib inline


In [ ]:
# load mndwi
mndwi_file_list = glob.glob('./datasets/indices/mndwi/mndwi*tif')
mndwi_file_list.sort()
dates = []
for d in mndwi_file_list:
    date = d[-14:-4]
    dates.append(date)
from datetime import datetime
dt = [datetime.strptime(date,'%Y-%m-%d').date() for date in dates]


In [ ]:
mndwi_file_list

In [ ]:
dates

In [ ]:
dt

In [ ]:
## convert tiff files to xarray dataset

#rasters = []
# for f in mndwi_file_list:
#     rasters.append(xr.open_dataset(f))
# ds = xr.concat(rasters, pd.to_datetime(dt))
# ds = ds.rename({'band_data':'mndwi','concat_dim':'time'})
# ds = ds.rio.write_crs(4326, inplace=True)

In [ ]:
dsW = xr.load_dataset('datasets/indices/mndwi_1987_2022.nc')

In [ ]:
dsW

In [ ]:
dsW.mndwi.mean('time').plot()

In [ ]:
dsW.mndwi.mean('time').plot(robust=True)

In [ ]:
dsV = xr.load_dataset('datasets/indices/ndvi_1987_2020.nc')

In [ ]:
dsV.ndvi.mean('time').plot()

In [ ]:
#A better visualisation
dsV.ndvi.mean('time').plot(robust=True)

In [ ]:
ds = xr.merge([dsW,dsV])

In [ ]:
ds

In [ ]:
ds.ndvi.mean('time').plot()

# What went wrong above???

In [ ]:
# dsV_test = dsV.rio.reproject_match(dsW)

In [ ]:
dsV = dsV.rio.write_crs(4326, inplace=True)
dsW = dsW.rio.write_crs(4326, inplace=True)

In [ ]:
dsV

In [ ]:
dsV2 = dsV.ndvi.data
dsV2 = np.squeeze(dsV2)
dsV2 = xr.DataArray(dsV2)
dsV2 = dsV2.rename({'dim_0':'time','dim_1': "y",'dim_2': "x"})
dsV2 = dsV2.assign_coords({
"time":dsV.time,
"x": dsV.x,
"y": dsV.y,
                        })
crs = dsV.rio.crs
dsV2 = dsV2.rio.write_crs(crs)
dsV2.name = 'ndvi'

In [ ]:
dsV2.mean('time').plot()

In [ ]:
dsW2 = dsW.mndwi.data
dsW2 = np.squeeze(dsW2)
dsW2 = xr.DataArray(dsW2)
dsW2 = dsW2.rename({'dim_0':'time','dim_1': "y",'dim_2': "x"})
dsW2 = dsW2.assign_coords({
"time":dsW.time,
"x": dsW.x,
"y": dsW.y,
                        })
crs = dsW.rio.crs
dsW2 = dsW2.rio.write_crs(crs)
dsW2.name = 'ndwi'

In [ ]:
dsV3 = dsV2.rio.reproject_match(dsW2)

In [ ]:
ds2 = xr.merge([dsW2,dsV3])

In [ ]:
ds2.ndvi.mean('time').plot()

In [ ]:
correlate = xr.corr(ds2.mndwi,ds2.ndvi, dim='time')

In [ ]:
correlate

In [ ]:
correlate.plot()

In [ ]:
from xrspatial.classify import natural_breaks

In [ ]:
classifiedV = natural_breaks(ds2.ndvi.mean('time'),5)
classifiedW = natural_breaks(ds2.mndwi.mean('time'),5)

In [ ]:
classifiedV.plot()

In [ ]:
classifiedW.plot()